# Points of Dispense - Template
Maxwell Kennady, Nora Murray, Elizabeth Speigle

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Optimization

Read in data from ...

In [2]:
distances = pd.read_csv('OD_Pairs_Distances.csv')
population = pd.read_excel('BG_master.xlsx')
bg_included = pd.read_csv('bg_pop_included.csv')

In [3]:
distances.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51700 entries, 0 to 51699
Data columns (total 6 columns):
block_group        51700 non-null int64
site_id            51700 non-null int64
DestinationRank    51700 non-null int64
TravelTime         51700 non-null float64
Miles              51700 non-null float64
Kilometers         51700 non-null float64
dtypes: float64(3), int64(3)
memory usage: 2.4 MB


In [9]:
distances.head()

,block_group,site_id,DestinationRank,TravelTime,Miles,Kilometers
0,1,1,14,21.779088,7.120065,11.456185
1,1,2,12,26.771438,6.972418,11.218621
2,1,3,23,27.973733,9.660235,15.543319
3,1,4,1,8.968479,3.010405,4.843741
4,1,5,7,19.793014,5.447165,8.764489


In [4]:
dist_miles = distances.pivot(index='block_group', columns='site_id', values='Miles')
dist_time = distances.pivot(index='block_group', columns='site_id', values='TravelTime')

In [6]:
dist_miles.head()

site_id,1,2,3,4,5,6,7,8,9,10,...,38,39,40,41,42,43,44,45,46,47
block_group,,,,,,,,,,,,,,,,,,,,,
1,7.120065,6.972418,9.660235,3.010405,5.447165,6.827666,4.073706,6.821528,15.200137,13.029100,...,11.882425,17.064616,7.135373,5.452286,11.044383,9.623167,15.717667,8.622107,7.038306,8.309050
2,7.164949,6.909107,9.571166,2.921336,5.383855,7.226126,4.010396,7.031647,15.111068,13.073984,...,11.793356,16.837082,7.072062,5.662405,10.955314,9.534098,15.927786,8.558796,6.730922,8.245740
3,7.575598,7.667544,10.018893,3.369063,6.323034,7.638033,4.949575,7.443554,15.558795,13.497286,...,12.241083,17.006874,6.975227,6.074312,11.403041,9.981825,16.339693,8.461960,6.106056,7.976724
4,8.066429,7.439514,10.399206,3.749376,5.914262,8.113201,4.540803,7.918722,15.939108,13.975464,...,12.621396,16.698807,6.667160,6.549480,11.783354,10.362138,16.814861,8.153893,5.797989,7.597902
5,6.621092,7.143660,9.495009,2.845178,5.788677,6.333064,4.415218,6.326926,15.034911,12.530127,...,11.717198,16.909133,7.579251,4.957684,10.879157,9.457940,15.223065,9.065985,7.135610,8.752929


In [8]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 8 columns):
BG_id         1100 non-null int64
STATEFP       1100 non-null int64
COUNTYFP      1100 non-null int64
TRACTCE       1100 non-null int64
BLKGRPCE      1100 non-null int64
POPULATION    1100 non-null int64
LATITUDE      1100 non-null float64
LONGITUDE     1100 non-null float64
dtypes: float64(2), int64(6)
memory usage: 68.9 KB


In [10]:
population.head()

,BG_id,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE
0,1,42,3,10300,1,2594,40.434602,-79.993347
1,2,42,3,10300,2,3336,40.436130,-79.990873
2,3,42,3,10300,3,353,40.437007,-79.982553
3,4,42,3,10300,4,317,40.437364,-79.977217
4,5,42,3,20100,1,1435,40.438538,-80.001596


In [12]:
bg_included.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 3 columns):
BG_id              1100 non-null int64
all_included       1100 non-null int64
random_included    1100 non-null float64
dtypes: float64(1), int64(2)
memory usage: 25.9 KB


In [13]:
bg_included.head()

,BG_id,all_included,random_included
0,1,1,0.588088
1,2,1,0.619573
2,3,1,0.457391
3,4,1,0.282319
4,5,1,0.283863


In [14]:
dist = dist_miles.values                     # can change to dist_time
N = population['POPULATION'].values
prop = bg_included['all_included'].values    # can change to random_included or other column

Create indices for block groups and PODs

In [20]:
blocks = range(len(N))
pods = range(len(dist[0]))

Initialize model for POD locations

In [23]:
m = gp.Model('POD_locations')

Using license file C:\Users\Elizabeth\gurobi.lic
Academic license - for non-commercial use only


Add decision variables x[i] for whether a POD is opened and y[i,j] for whether POD i serves block group j

In [24]:
x = m.addVars(pods, vtype=GRB.BINARY, name='x')
y = m.addVars(pods, blocks, vtype=GRB.BINARY, name='x')

Set up objective function to minimize total distance across the population

In [25]:
obj = gp.quicksum(dist[j,i] * x[i] * y[i, j] * N[j] * prop[j] 
                  for j in blocks for i in pods)
m.setObjective(obj, GRB.MINIMIZE)

Constraint: y[i,j] can only be 1 if x[i] is also 1, meaning POD i is opened

In [26]:
m.addConstrs((y[i,j] <= x[i] for i in pods for j in blocks), name='y_if_x')

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

Constraint: each block group must be assigned one shelter

In [27]:
m.addConstrs((gp.quicksum(y[i, j] for i in pods) == 1
             for j in blocks), name='all_blocks_assigned')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

Constraint: number of PODs opened must be ...

In [28]:
m.addConstr((gp.quicksum(x[i] for i in pods) <= 47), name='pods_opened')

<gurobi.Constr *Awaiting Model Update*>

Optimize model

In [29]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 52801 rows, 51747 columns and 155147 nonzeros
Model fingerprint: 0x8d9a9180
Model has 51606 quadratic objective terms
Variable types: 0 continuous, 51747 integer (51747 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+00, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 1.881399e+07
Presolve removed 1 rows and 0 columns
Presolve time: 0.20s
Presolved: 104406 rows, 103353 columns, 309918 nonzeros
Variable types: 0 continuous, 103353 integer (103353 binary)
Found heuristic solution: objective 2763511.3367

Explored 1 nodes (0 simplex iterations) in 1.10 seconds
Thread count was 12 (of 12 available processors)

Solution count 2: 2.76351e+06 1.8814e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.763511336736e+06, best bound 2.763511336736e+06, gap 0.0000%


### Analysis

Which block groups were assigned to which shelters?

In [30]:
block_pod_list = [[i,j] for i in pods for j in blocks if y[i, j].x==1]

In [31]:
block_pod_list # can export to view in a map

[[0, 459],
 [0, 469],
 [0, 471],
 [0, 472],
 [0, 475],
 [0, 476],
 [0, 478],
 [0, 479],
 [0, 480],
 [0, 483],
 [0, 484],
 [0, 1058],
 [0, 1093],
 [0, 1097],
 [1, 689],
 [1, 690],
 [1, 693],
 [1, 695],
 [1, 696],
 [1, 697],
 [1, 698],
 [1, 699],
 [1, 709],
 [1, 710],
 [1, 711],
 [1, 712],
 [1, 713],
 [1, 714],
 [1, 715],
 [1, 716],
 [1, 717],
 [1, 718],
 [1, 767],
 [1, 772],
 [1, 775],
 [1, 776],
 [2, 655],
 [2, 656],
 [2, 657],
 [2, 658],
 [2, 659],
 [2, 661],
 [2, 662],
 [2, 664],
 [2, 665],
 [2, 666],
 [2, 667],
 [2, 674],
 [2, 675],
 [2, 676],
 [2, 677],
 [2, 678],
 [2, 679],
 [2, 781],
 [3, 0],
 [3, 1],
 [3, 2],
 [3, 3],
 [3, 4],
 [3, 5],
 [3, 6],
 [3, 7],
 [3, 8],
 [3, 9],
 [3, 11],
 [3, 12],
 [3, 25],
 [3, 32],
 [3, 162],
 [3, 166],
 [3, 167],
 [3, 168],
 [3, 169],
 [3, 170],
 [3, 171],
 [3, 172],
 [3, 173],
 [3, 174],
 [3, 175],
 [3, 176],
 [3, 177],
 [3, 178],
 [3, 179],
 [3, 180],
 [3, 181],
 [3, 182],
 [3, 183],
 [3, 184],
 [3, 185],
 [3, 186],
 [3, 187],
 [3, 188],
 [3, 189]